<a href="https://colab.research.google.com/github/Phavour-EBEN/Personal-Deep-Learning-Lessons/blob/main/Intro_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tensorflow==2.15.0 tensorflow-hub keras==2.15.0

Introduction to Natural Language Processing
* Getting the necessary doc and files

In [ ]:
# !wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
# !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

--2024-12-11 20:30:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.212.123, 142.250.200.251, 172.217.18.59, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.212.123|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: 'nlp_getting_started.zip'

     0K .......... .......... .......... .......... ..........  8%  145K 4s
    50K .......... .......... .......... .......... .......... 16%  280K 3s
   100K .......... .......... .......... .......... .......... 25%  531K 2s
   150K .......... .......... .......... .......... .......... 33%  559K 1s
   200K .......... .......... .......... .......... .......... 42%  721K 1s
   250K .......... .......... .......... .......... .......... 50%  667K 1s
   300K .......... .......... .......... .......... .......... 59%  381K 1s
   350K .......... .......... ..........

In [4]:
from helper_functions import unzip_data, plot_loss_curves, create_tensorboard_callback, compare_historys
# unzip_data('nlp_getting_started.zip')

In [5]:
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'train.csv')
test_data = pd.read_csv(r'test.csv')
train_data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# shuffling the train_data
train_data = train_data.sample(frac=1, random_state=42)
train_data.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
train_data.target.value_counts()

,count
target,
0,4342
1,3271


In [8]:
len(train_data), len(test_data)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0, len(train_data)-5)

for row in train_data[['text', 'target']][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target :{target} ", "(real diaster)" if target>0 else "(not real diseater)")
    print(f"Text :{text} \n")
    print('......\n')

Target :1  (real diaster)
Text :#News: 'Many deaths' in shipwreck: Rescuers are trying to save hundreds of migrants after the... http://t.co/tX51oYbrN6 via @TheNewsHype 

......

Target :0  (not real diseater)
Text :matako_3: Breaking news! Unconfirmed! I just heard a loud bang nearby. in what appears to be a blast of wind from my neighbour's ass. 

......

Target :0  (not real diseater)
Text :@thoutaylorbrown I feel like accidents are just drawn to you but I'm happy you've survived all of them. Hope you're okay!!! 

......

Target :0  (not real diseater)
Text :@XGN_Infinity @Ronin_Carbon HAHAH Mutual host preset Bal no nades radar on = demolish 

......

Target :0  (not real diseater)
Text :Large rain drops falling in Rock Hill off Anderson Road. #rain #scwx #drought 

......



Spliting data into training, testing anad validation sets

In [10]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_data['text'].to_numpy(),
                                                                            train_data['target'].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [11]:
len(train_sentences), len(val_sentences), len(train_labels), len(val_labels)

(6851, 762, 6851, 762)

In [12]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

**converting text to numbers**
* Tokenization: direct mapping of tekens(a token could be a word or a character)
* Embedding: creating a metrix of feature vectors for each token(s)(the size of the feature vector can be define and the embedding can be learned)

In [13]:
# Tokenization
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

In [14]:
text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize = 'lower_and_strip_punctuation',
                                    split = 'whitespace',
                                    ngrams= None,
                                    output_mode = 'int',
                                    output_sequence_length = None,
                                    pad_to_max_tokens = False)

In [15]:
len(train_sentences[0].split())

7

In [16]:
# finding the number of avg word in ine training sentences
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
# setting up text vectorizer
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    output_mode = 'int',
    output_sequence_length = max_length
)

In [18]:
text_vectorizer.adapt(train_sentences)

In [19]:
# creating a smaple sentence tokens

sentence = "This is a sample sentence statement"
text_vectorizer([sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  19,    9,    3, 8839,    1, 2283,    0,    0,    0,    0,    0,
           0,    0,    0,    0]])>

In [20]:
# choosing the random sentence in the training set
random_sentence = random.choice(train_sentences)
print(f"Origin text: \n{random_sentence}\n")

text_vectorizer([random_sentence])

Origin text: 
#charminar demolish if it in falling state anyway take engineers opinion
#Telangana



<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1,  516,   47,   15,    4, 1140,  265, 2557,  167,    1, 2824,
        7740,    0,    0,    0]])>

In [21]:
# getting the unique characters in the vocabulary
word_in_vocab = text_vectorizer.get_vocabulary()
top_5 = word_in_vocab[:5]
buttom_5 = word_in_vocab[-5:]
print(len(word_in_vocab))
print(top_5)
print(buttom_5)

10000
['', '[UNK]', 'the', 'a', 'in']
['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [22]:
# Setting the embedding space
from tensorflow.keras.layers import Embedding
embedding = Embedding(
    input_dim = max_vocab_length,
    input_length = max_length,
    output_dim = 128
)

In [23]:
# getting random number of embedding
print(f"Original sentences :\n{random_sentence}\n")
print(f"Embedded sentences : \n")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original sentences :
#charminar demolish if it in falling state anyway take engineers opinion
#Telangana

Embedded sentences : 



<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.0361232 ,  0.01485253, -0.00162486, ...,  0.03257692,
          0.00717517, -0.02611988],
        [-0.04337193, -0.04627571,  0.00488346, ...,  0.02086611,
          0.04972548,  0.02134205],
        [ 0.04693193,  0.00424787,  0.04869466, ...,  0.03882147,
          0.02753821,  0.01995939],
        ...,
        [ 0.00682262,  0.01260522, -0.03599947, ...,  0.02013713,
         -0.01171602, -0.04958178],
        [ 0.00682262,  0.01260522, -0.03599947, ...,  0.02013713,
         -0.01171602, -0.04958178],
        [ 0.00682262,  0.01260522, -0.03599947, ...,  0.02013713,
         -0.01171602, -0.04958178]]], dtype=float32)>

In [24]:
# checking out a single token
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.0361232 ,  0.01485253, -0.00162486,  0.02885953, -0.03275198,
        0.03249994, -0.03537705,  0.02467126, -0.01099426,  0.00541434,
        0.0165887 ,  0.02685462, -0.00809263, -0.0319424 ,  0.00314573,
       -0.03699076, -0.03551867, -0.04070885,  0.01928035, -0.02687637,
       -0.00983026, -0.02156744,  0.03314171,  0.04778915, -0.04749955,
       -0.01360793,  0.00450714,  0.03589344,  0.02018182, -0.0464371 ,
        0.00080021, -0.0232308 ,  0.02617754, -0.02166853,  0.03347183,
       -0.00134442, -0.01919013,  0.03743822, -0.01878663,  0.00168443,
        0.04064138, -0.02627735, -0.02343726,  0.03272427,  0.01351161,
       -0.02522492, -0.01076503,  0.00794167, -0.02598373, -0.01348935,
       -0.02016897,  0.01032031, -0.02141529, -0.03838154,  0.00711534,
       -0.02398723,  0.04378538, -0.04908261, -0.01342579, -0.04121602,
       -0.00594439,  0.02764655,  0.02487475,  0.02860047,  0.02386869,
        0.033179

Modelling text datase(running a series of expperiment)
* First Baseline

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [26]:

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# fitting the model
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Model 0 score: {baseline_score*100:.2f}")

Model 0 score: 79.27


In [28]:
baseline_predictions = model_0.predict(val_sentences)
baseline_predictions[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

In [29]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

**creating a function to keep track of our model accuracies**
* F1 scores
* precision
* accuracy
* recall


In [30]:
# from sklearn.metrics import accuracy_score, recall, f1_score, precision
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_accuracy(y_true, y_pred):
    model_accuracy = accuracy_score(y_true, y_pred)*100
    model_precision, model_recall, model_f1_score, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    model_results = {
        "model accuracy": model_accuracy,
        "model precision": model_precision*100,
        "model recall": model_recall*100,
        "model f1 score": model_f1_score*100
    }

    return model_results



In [31]:
baseline_results = calculate_accuracy(val_labels, baseline_predictions)
baseline_results

{'model accuracy': 79.26509186351706,
 'model precision': 81.11390004213173,
 'model recall': 79.26509186351706,
 'model f1 score': 78.6218975804955}

model 2 Dense model(Feed forward Neural Network)

In [32]:
logs_dir = 'model logs'

In [33]:
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer='uniform',
                                     input_length = max_length)

inputs = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = model_2_embedding(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model_2 = tf.keras.Model(inputs, output, name='model_2')

model_2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (  (None, 128)               0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1280129 (4.88 MB)
Trainable params: 1280129 (

In [34]:
from helper_functions import create_tensorboard_callback
model_2.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)



In [35]:
train_sentences.shape, train_labels.shape, val_sentences.shape, val_labels.shape

((6851,), (6851,), (762,), (762,))

In [36]:
model_2_history = model_2.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name = 'model_2')]
)

Saving TensorBoard log files to: model logs/model_2/20250107-005324
Epoch 1/5
215/215 [==============================] - 10s 34ms/step - loss: 0.6139 - accuracy: 0.6916 - val_loss: 0.5358 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.4412 - accuracy: 0.8181 - val_loss: 0.4759 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.3460 - accuracy: 0.8603 - val_loss: 0.4582 - val_accuracy: 0.7887
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.2839 - accuracy: 0.8889 - val_loss: 0.4655 - val_accuracy: 0.7927
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2379 - accuracy: 0.9129 - val_loss: 0.4834 - val_accuracy: 0.7874


In [37]:
model_2.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4834 - accuracy: 0.7874


[0.4834246337413788, 0.787401556968689]

In [38]:
model_2_pred_prop = model_2.predict(val_sentences)
model_2_pred_prop.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [39]:
model_2_pred_prop[:10]

array([[0.31810108],
       [0.7398806 ],
       [0.99728394],
       [0.11788929],
       [0.11450759],
       [0.93156785],
       [0.898836  ],
       [0.99349654],
       [0.964534  ],
       [0.2634395 ]], dtype=float32)

In [40]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [41]:
# model prediction propability to labels
model_pred = tf.squeeze(tf.round(model_2_pred_prop))
model_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [42]:
model_2_results = calculate_accuracy(val_labels, model_pred.numpy())
model_2_results

{'model accuracy': 78.74015748031496,
 'model precision': 79.42180127180873,
 'model recall': 78.74015748031496,
 'model f1 score': 78.38012115396069}

In [43]:
import numpy as np
np.array(list(model_2_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Visualizing the embedding

In [44]:
word_in_vocab = text_vectorizer.get_vocabulary()
print(len(word_in_vocab))
word_in_vocab[-5:]

10000


['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']

In [45]:
embed_weights = model_2.get_layer('embedding_1').get_weights()[0]
embed_weights

array([[ 0.02820506,  0.01558898,  0.00448632, ...,  0.02646688,
         0.05135892,  0.06321478],
       [ 0.05977183,  0.0495188 ,  0.04239571, ...,  0.00781888,
         0.04437122, -0.03599814],
       [ 0.0084357 , -0.00601062, -0.02231671, ..., -0.00868563,
         0.02123116, -0.02730276],
       ...,
       [-0.03995756, -0.01846583,  0.03073475, ..., -0.00494412,
         0.02174694,  0.00774695],
       [ 0.04402622,  0.05764099,  0.04788615, ..., -0.00999798,
         0.03577803, -0.00143235],
       [ 0.06621356,  0.00873781,  0.05315956, ..., -0.0764842 ,
         0.06485543,  0.04445469]], dtype=float32)

In [46]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(word_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

model 3(Recurrent neural networks)

In [47]:
model_3_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer='uniform',
                                     input_length = max_length)

inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = model_3_embedding(x)
# x = layers.LSTM(64, return_sequences=True)(x) // for stacking multiple rrn(S)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs, outputs,name="model_3")
model_3.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [48]:
# compiling the model
model_3.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [49]:
# fittng the model
model_3.fit(
    train_sentences,
    train_labels,
    epochs= 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name='model_3')]
)

Saving TensorBoard log files to: model logs/model_3/20250107-005344
Epoch 1/5
215/215 [==============================] - 11s 41ms/step - loss: 0.5084 - accuracy: 0.7484 - val_loss: 0.4599 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3142 - accuracy: 0.8720 - val_loss: 0.4663 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2137 - accuracy: 0.9223 - val_loss: 0.5764 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1558 - accuracy: 0.9418 - val_loss: 0.6813 - val_accuracy: 0.7572
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1045 - accuracy: 0.9603 - val_loss: 0.7814 - val_accuracy: 0.7598


In [50]:
model_3_pred_prop = model_3.predict(val_sentences)
model_3_pred = tf.squeeze(tf.round(model_3_pred_prop))
print(model_3_pred[:10])
print(val_labels[:10])

24/24 [==============================] - 0s 3ms/step
tf.Tensor([0. 1. 1. 0. 0. 1. 0. 1. 1. 0.], shape=(10,), dtype=float32)
[0 0 1 1 1 1 1 1 1 0]


In [51]:
calculate_accuracy(val_labels, model_3_pred.numpy())

{'model accuracy': 75.98425196850394,
 'model precision': 76.15439957650938,
 'model recall': 75.98425196850394,
 'model f1 score': 75.74575318690255}

Bulding a Gated Recurrent Neural Network(GRU) model

In [52]:
model_4_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer='uniform',
                                     input_length = max_length)

inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = model_4_embedding(x)
x = layers.GRU(64)(x)
# x = layers.LSTM(32, return_sequences=True)(x)
# x = layers.GRU(16, return_sequences=True)(x)
# x = layers.GRU(16)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = 'model_4')

model_4.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_3 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1317313 (5.03 MB)
Trainable params: 1317313 (5.03 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [53]:
model_4.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [54]:
model_4_history = model_4.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name ='model_4')]
)

Saving TensorBoard log files to: model logs/model_4/20250107-005407
Epoch 1/5
215/215 [==============================] - 13s 50ms/step - loss: 0.5245 - accuracy: 0.7357 - val_loss: 0.4528 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3140 - accuracy: 0.8672 - val_loss: 0.5139 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2189 - accuracy: 0.9155 - val_loss: 0.5208 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1564 - accuracy: 0.9457 - val_loss: 0.5294 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1195 - accuracy: 0.9597 - val_loss: 0.6256 - val_accuracy: 0.7717


In [55]:
model_4_pred_prop=  model_4.predict(val_sentences)
model_4_pred=  tf.squeeze(tf.round(model_4_pred_prop))
model_4_pred[:5]

24/24 [==============================] - 0s 3ms/step


<tf.Tensor: shape=(5,), dtype=float32, numpy=array([0., 1., 1., 0., 0.], dtype=float32)>

In [56]:
calculate_accuracy(val_labels, model_4_pred.numpy())

{'model accuracy': 77.16535433070865,
 'model precision': 77.91293682827462,
 'model recall': 77.16535433070865,
 'model f1 score': 76.72567495577469}

In [57]:
import numpy as np
np.array(list(calculate_accuracy(val_labels, model_4_pred.numpy()).values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

Building a bidirectional RNN

In [58]:
from tensorflow.keras import layers

model_5_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embeddings_initializer='uniform',
                                     input_length = max_length)

inputs_5 = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs_5)
x = model_5_embedding(x)

x = layers.Bidirectional(layers.LSTM(64))(x)
outputs_5 = layers.Dense(1, activation='sigmoid')(x)
model_5 = tf.keras.Model(inputs_5, outputs_5, name='model_5')

In [59]:
model_5.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_4 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirection  (None, 128)               98816     
 al)                                                             
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1378945 (5.26 MB)
Trainable params: 1378945 (

In [60]:
model_5.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [61]:
model_5.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name ='model_5')]
)

Saving TensorBoard log files to: model logs/model_5/20250107-005428
Epoch 1/5
215/215 [==============================] - 12s 37ms/step - loss: 0.5169 - accuracy: 0.7481 - val_loss: 0.4555 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.3162 - accuracy: 0.8718 - val_loss: 0.4886 - val_accuracy: 0.7808
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2138 - accuracy: 0.9228 - val_loss: 0.5628 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1504 - accuracy: 0.9464 - val_loss: 0.6513 - val_accuracy: 0.7546
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1077 - accuracy: 0.9613 - val_loss: 0.7318 - val_accuracy: 0.7598


In [62]:
model_5_pred_prop = model_5.predict(val_sentences)
model_5_pred_prop = model_5_pred_prop.reshape(-1)
model_5_pred_prop[:10]

model_5_pred = tf.squeeze(tf.round(model_5_pred_prop))

model_5_pred[:10]

24/24 [==============================] - 1s 3ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [63]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [64]:
calculate_accuracy(val_labels, model_5_pred.numpy())

{'model accuracy': 75.98425196850394,
 'model precision': 75.94780880165999,
 'model recall': 75.98425196850394,
 'model f1 score': 75.90768821741806}

Convolutional neural network for text

`* Input text => tokeniation => embedding => Conv1D layer => Pool layer => Dense layer => Output layer`


In [65]:
embedding_test = embedding(text_vectorizer(['I love deep learning']))
conv_1d = layers.Conv1D(filters=32,
                        kernel_size=5,
                        activation='relu',
                        padding='valid')
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

In [66]:
# creating a Conv1D for our proble
model_6_embedding = layers.Embedding(input_dim = max_vocab_length,
                                    output_dim = 128,
                                    embeddings_initializer='uniform',
                                    input_length = max_length)

inputs_6 = layers.Input((1,), dtype=tf.string)
x = text_vectorizer(inputs_6)
x = model_6_embedding(x)
x = layers.Conv1D(filters=64,
                  kernel_size=3,
                  strides = 1,
                  activation='relu',
                  padding = 'same')(x)
x = layers.GlobalMaxPool1D()(x)
outputs_6 = layers.Dense(1, activation='sigmoid')(x)
model_6 = tf.keras.Model(inputs_6, outputs_6, name = 'model_5')
model_6.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (Text  (None, 15)                0         
 Vectorization)                                                  
                                                                 
 embedding_5 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_1 (Conv1D)           (None, 15, 64)            24640     
                                                                 
 global_max_pooling1d_1 (Gl  (None, 64)                0         
 obalMaxPooling1D)                                               
                                                                 
 dense_4 (Dense)             (None, 1)                 65  

In [67]:
model_6.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [68]:
history_6 = model_6.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name ='model_6')]
)

Saving TensorBoard log files to: model logs/model_6/20250107-005454
Epoch 1/5
215/215 [==============================] - 9s 36ms/step - loss: 0.5630 - accuracy: 0.7183 - val_loss: 0.4767 - val_accuracy: 0.7887
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3475 - accuracy: 0.8540 - val_loss: 0.4678 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2192 - accuracy: 0.9172 - val_loss: 0.5417 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.1380 - accuracy: 0.9523 - val_loss: 0.6288 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0951 - accuracy: 0.9691 - val_loss: 0.7052 - val_accuracy: 0.7782


In [69]:
model_6_pred_prop = model_6.predict(val_sentences)
model_6_pred = tf.squeeze(tf.round(model_6_pred_prop))
model_6_pred[:10]

24/24 [==============================] - 0s 3ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [70]:
calculate_accuracy(val_labels, model_6_pred.numpy())

{'model accuracy': 77.82152230971128,
 'model precision': 78.33771178803303,
 'model recall': 77.82152230971128,
 'model f1 score': 77.48637513129985}

Using a pre-trained model from tensorflow Hub


In [71]:
import tensorflow_hub as hub

In [72]:
'''
TensorFlow Hub Pretrained Sentence Encoder
The main difference between the embedding layer we created and the Universal Sentence Encoder is that rather than create a word-level embedding, the Universal Sentence Encoder, as you might've guessed, creates a whole sentence-level embedding.
'''
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

embed_sample = embed([random_sentence, "I love tensorflow"])
print(embed_sample[0][:10])

tf.Tensor(
[ 0.05226733  0.03346372  0.03949298  0.04812825 -0.00825532  0.04315928
  0.01529975  0.0590469  -0.01334779 -0.02981975], shape=(10,), dtype=float32)


In [73]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-large/5",
                                        trainable = False,
                                        input_shape = [],
                                        dtype = tf.string)


In [74]:
sentence_encoder_layer

In [78]:
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
],name  = 'model_7')


model_7.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               147354880 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 147387777 (562.24 MB)
Trainable params: 32897 (128.50 KB)
Non-trainable params: 147354880 (562.11 MB)
_________________________________________________________________


In [79]:
model_7.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [80]:
history_7 = model_7.fit(
    train_sentences,
    train_labels,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name ='model_7')]
)

Saving TensorBoard log files to: model logs/model_7/20250107-005852
Epoch 1/5
215/215 [==============================] - 37s 107ms/step - loss: 0.4912 - accuracy: 0.7965 - val_loss: 0.4245 - val_accuracy: 0.8150
Epoch 2/5
215/215 [==============================] - 14s 66ms/step - loss: 0.3986 - accuracy: 0.8269 - val_loss: 0.4174 - val_accuracy: 0.8150
Epoch 3/5
215/215 [==============================] - 11s 53ms/step - loss: 0.3819 - accuracy: 0.8333 - val_loss: 0.4180 - val_accuracy: 0.8110
Epoch 4/5
215/215 [==============================] - 10s 49ms/step - loss: 0.3729 - accuracy: 0.8396 - val_loss: 0.4160 - val_accuracy: 0.8176
Epoch 5/5
215/215 [==============================] - 13s 61ms/step - loss: 0.3643 - accuracy: 0.8435 - val_loss: 0.4152 - val_accuracy: 0.8110


In [81]:
model_7_pred_prop = model_7.predict(val_sentences)
model_7_pred = tf.squeeze(tf.round(model_7_pred_prop))
model_7_pred[:10]

24/24 [==============================] - 6s 45ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [82]:
calculate_accuracy(val_labels, model_7_pred.numpy())

{'model accuracy': 81.10236220472441,
 'model precision': 81.36960942796058,
 'model recall': 81.10236220472441,
 'model f1 score': 80.93098281743121}

In [83]:
baseline_results

{'model accuracy': 79.26509186351706,
 'model precision': 81.11390004213173,
 'model recall': 79.26509186351706,
 'model f1 score': 78.6218975804955}

Creating 10% of the data and clone the pre-trained model on the dataset

In [85]:
train_90_percent, train_10_percent, train_labels_90_percent, val_labels_10_percent = train_test_split(train_sentences, train_labels, test_size=0.1, random_state=42)
len(train_10_percent), len(train_sentences)

(686, 6851)

In [93]:
pd.Series(val_labels_10_percent).value_counts()

,count
0,415
1,271


In [87]:
# using the clone method from keras.model instance
model_8 = tf.keras.models.clone_model(model_7)

# compiling and fitting the model
model_8.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

model_8.fit(
    train_10_percent,
    val_labels_10_percent,
    epochs = 5,
    validation_data = (val_sentences, val_labels),
    callbacks = [create_tensorboard_callback(logs_dir, experiment_name ='model_8')]
)

Saving TensorBoard log files to: model logs/model_8/20250107-011447
Epoch 1/5
22/22 [==============================] - 30s 312ms/step - loss: 0.6586 - accuracy: 0.6866 - val_loss: 0.6380 - val_accuracy: 0.7257
Epoch 2/5
22/22 [==============================] - 3s 123ms/step - loss: 0.5825 - accuracy: 0.8280 - val_loss: 0.5780 - val_accuracy: 0.7598
Epoch 3/5
22/22 [==============================] - 3s 137ms/step - loss: 0.5057 - accuracy: 0.8586 - val_loss: 0.5225 - val_accuracy: 0.7966
Epoch 4/5
22/22 [==============================] - 2s 106ms/step - loss: 0.4427 - accuracy: 0.8615 - val_loss: 0.4838 - val_accuracy: 0.8058
Epoch 5/5
22/22 [==============================] - 2s 104ms/step - loss: 0.3959 - accuracy: 0.8703 - val_loss: 0.4672 - val_accuracy: 0.8031


In [88]:
model_8_pred_prop = model_8.predict(val_sentences)
model_8_pred = tf.squeeze(tf.round(model_8_pred_prop))
model_8_pred[:10]

24/24 [==============================] - 8s 45ms/step


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [89]:
calculate_accuracy(val_labels, model_8_pred.numpy())

{'model accuracy': 80.31496062992126,
 'model precision': 80.87509988853203,
 'model recall': 80.31496062992126,
 'model f1 score': 80.03810160670334}

In [90]:
baseline_results

{'model accuracy': 79.26509186351706,
 'model precision': 81.11390004213173,
 'model recall': 79.26509186351706,
 'model f1 score': 78.6218975804955}